In [ ]:
from functools import partial
import pandas as pd
import os
from pathlib import Path

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
import ipyvuetify as v

from utils import messages as ms
from utils import parameters as pm
from scripts import vizualisation as viz
from scripts import table as table_run

In [ ]:
#input and output as mutable objects 
class Viz_io:
    def __init__(self):
        #input
        self.sources = None
        self.bands = None

viz_io = Viz_io()

In [ ]:
#output 
viz_output = sw.Alert().add_msg(ms.VIZ_INTRO)

#btn 
viz_btn = sw.Btn(ms.VIZ_BTN)

#create the inputs 
viz_sources = v.Select(items=pm.sources, label=ms.VIZ_SOURCES, v_model=None, multiple=True)
viz_bands = v.Select(items=[*pm.getAvailableBands()], label=ms.VIZ_BANDS, v_model=None)

#bind the inputs
viz_output.bind(viz_sources, viz_io, 'sources')
viz_output.bind(viz_bands, viz_io, 'bands')

#create a band selector 
id_ = 'viz_widget'
title = 'select your vizualisation Parameters'

viz_input = sw.Tile(
    id_,
    title,
    btn=viz_btn,
    inputs=[viz_sources, viz_bands],
    output=viz_output
)

In [ ]:
#vizualise data 
#def process_start(widget, event, data, output, m, maps):
def process_start(widget, event, data, output):
    
    #toggle the loading button
    widget.toggle_loading()
    
    #load the input 
    file = getattr(tb_io, 'file')
    pts = getattr(tb_io, 'pts')
    bands = getattr(viz_io, 'bands')
    sources = getattr(viz_io, 'sources')
    
    #check input
    if not output.check_input(file, ms.NO_PTS): return widget.toggle_loading()
    if not output.check_input(bands, ms.NO_BANDS): return widget.toggle_loading()
    if not output.check_input(sources, ms.NO_SOURCES): return widget.toggle_loading()
    
    #security when user remove all satellites (sources = [])
    if not len(sources) > 0: 
        output.add_msg(ms.NO_SOURCES, 'error')
        return widget.toggle_loading()
    
    #generate a sum-up of the inputs
    msg = viz.set_msg(pts, bands, ' & '.join(sources), Path(file).stem)
    
    output.add_msg(msg, 'warning')
    
    #toggle the loading button
    widget.toggle_loading()
    
    return 

viz_btn.on_event('click', partial(
    process_start,
    output = viz_output
))   

In [ ]:
viz_input